In [4]:
%cd ..

/home/soda/rcappuzz/work/benchmark-join-suggestions


In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import pickle
import polars as pl

In [21]:
import src.table_integration.join_profiling as jp

In [22]:
with open("generated_candidates.pickle", "rb") as fp:
    generated_candidates = pickle.load(fp)

In [23]:
one_candidate = generated_candidates["minhash"].popitem()
hash_, mdata = one_candidate
source_metadata = mdata.source_metadata
candidate_metadata = mdata.candidate_metadata
source_table = pl.read_parquet(source_metadata["full_path"])
candidate_table = pl.read_parquet(candidate_metadata["full_path"])
left_on = mdata.left_on
right_on = mdata.right_on


In [24]:
mdata.similarity_score

40

In [25]:
unique_source = jp.find_unique_keys(source_table, left_on)
unique_cand = jp.find_unique_keys(candidate_table, right_on)

In [26]:
jp.measure_cardinality_proportion(source_table, candidate_table, left_on, right_on)

0.1605925925925926

The cardinality proportion is larger than 1!!! This is not something they are accounting for in the heuristic. 

How are we handling these situations? Should we go for `min([card_source/card_cand, card_cand/card_source])` instead? Is this even meaningful? 

I am having issues here to find the containment 

In [27]:
unique_source = source_table[left_on].unique()
unique_cand = candidate_table[right_on].unique()
intersection=source_table.join(unique_cand, left_on=left_on, right_on=right_on, how="inner")
containment = len(intersection)/len(source_table)
print(containment)

0.28004152608357125


In [28]:
jp.measure_join_quality(source_table, candidate_table, left_on, right_on)

2

In [30]:
jp.estimate_join_size(source_table, candidate_table, left_on, right_on)

[30729.55376910912, 4934.938708994709]

In [45]:
tot_dict = []
for index_name, candidates in generated_candidates.items():
    print(index_name)
    
    for hash_, mdata in candidates.items():
        prof_dict = {}
        source_metadata = mdata.source_metadata
        candidate_metadata = mdata.candidate_metadata
        print(candidate_metadata["df_name"])
        source_table = pl.read_parquet(source_metadata["full_path"])
        candidate_table = pl.read_parquet(candidate_metadata["full_path"])
        left_on = mdata.left_on
        right_on = mdata.right_on
        estimates = jp.estimate_join_size(source_table, candidate_table, left_on, right_on)
        print(f"Estimated join size: {min(estimates):.2f}")
        join_quality = jp.measure_join_quality(source_table, candidate_table, left_on, right_on)
        print(f"Join quality: {join_quality:.2f}")
        containment=jp.measure_containment(source_table, candidate_table, left_on, right_on)
        print(f"Containment: {containment:.2f}")
        card_prop = jp.measure_cardinality_proportion(source_table, candidate_table, left_on, right_on)
        print(f"Cardinality proportion: {card_prop:.2f}")
        if min(estimates) < 1e6:
            joined =  jp.merge_table(source_table, candidate_table, left_on=left_on, right_on=right_on, how="inner")
            print("True join size: ", len(joined))
        else:
            joined = None
        prof_dict["candidate_id"] = mdata.candidate_id
        prof_dict["index"] = index_name
        prof_dict["source_table"] = source_metadata["full_path"]
        prof_dict["source_name"] = source_metadata["df_name"]
        prof_dict["candidate_table"] = candidate_metadata["full_path"]
        prof_dict["candidate_name"] = candidate_metadata["df_name"]
        # prof_dict["left_on"] = left_on
        # prof_dict["right_on"] = right_on
        # prof_dict["size_estimates"] = estimates
        prof_dict["min_estimate"] = min(estimates)
        prof_dict["join_quality"] = join_quality
        prof_dict["containment"] = containment
        prof_dict["cardinality_proportion"] = card_prop
        if joined is not None:        
            prof_dict["true_join_size"] = len(joined)
        else:
            prof_dict["true_join_size"] = 0
        
        tot_dict.append(prof_dict)


lazo
movie-revenues-target
Estimated join size: 3888.54
Join quality: 4.00
Containment: 1.00
Cardinality proportion: 1.00
True join size:  3951
minhash
wordnet_artist_actedIn_hasWonPrize
Estimated join size: 4341.48
Join quality: 0.00
Containment: 0.06
Cardinality proportion: 1.85
True join size:  540
wordnet_person_wasCreatedOnDate_directed
Estimated join size: 2996.89
Join quality: 0.00
Containment: 0.03
Cardinality proportion: 1.29
True join size:  115
wordnet_artist_actedIn_hasWebsite
Estimated join size: 3792.07
Join quality: 0.00
Containment: 0.07
Cardinality proportion: 1.32
True join size:  315
wordnet_album_isLocatedIn_wasCreatedOnDate
Estimated join size: 2823.23
Join quality: 1.00
Containment: 0.16
Cardinality proportion: 1.43
True join size:  654
movie-revenues-target
Estimated join size: 3888.54
Join quality: 4.00
Containment: 1.00
Cardinality proportion: 1.00
True join size:  3951
wordnet_person_hasWebsite_directed
Estimated join size: 4076.39
Join quality: 0.00
Containme

In [47]:
prof_df = pl.from_dicts(tot_dict)

In [49]:
prof_df = prof_df.drop(
    [
        "source_table",
        "candidate_table"
    ]
)

In [50]:
prof_df

candidate_id,source_name,candidate_name,min_estimate,join_quality,containment,cardinality_proportion,true_join_size
str,str,str,f64,i64,f64,f64,i64
"""6e39d88898326d…","""movies-prepare…","""movie-revenues…",3888.54428,4,1.0,1.0,3951
"""10bf06ccabc7b7…","""movies-prepare…","""wordnet_artist…",4341.479968,0,0.060358,1.845331,540
"""878eb7ed18fbca…","""movies-prepare…","""wordnet_person…",2996.890617,0,0.030311,1.287411,115
"""5818aaace07339…","""movies-prepare…","""wordnet_artist…",3792.066948,0,0.068002,1.321491,315
"""a781e57133964c…","""movies-prepare…","""wordnet_album_…",2823.231418,1,0.163943,1.431698,654
"""6e39d88898326d…","""movies-prepare…","""movie-revenues…",3888.54428,4,1.0,1.0,3951
"""30d041a8969c64…","""movies-prepare…","""wordnet_person…",4076.392021,0,0.084871,0.672695,351
"""5eaf1b8a2c4cca…","""movies-prepare…","""yago_seltab_wo…",179690.935045,0,0.021349,0.653687,971
"""2961999f4ba187…","""movies-prepare…","""wordnet_office…",4668.651963,0,0.016342,0.80076,65
